# Respostas Questionário IV

**Aluno:** Luiz Fernando Rabelo (11796893)

## Bibliotecas Utilizadas

Para a resolução do questionário, foram utilizadas as bibliotecas _numpy_, _pandas_, _matplotlib_, _seaborn_, e _sklearn_, as quais são importadas abaixo:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

## Leitura dos Dados de Treinamento e Teste

O primeiro passo a ser tomado é a leitura dos dados de treinamento e teste, os quais contam com os seguintes atributos:

- _PassengerId:_ número de identificação do passageiro
- _Survived:_ sobrevivência do passageiro (0 = não | 1 = sim)
- _Pclass:_ classe da passagem (1 = primeira, 2 = segunda ou 3 = terceira)
- _Name:_ nome do passageiro
- _Sex:_ sexo do passageiro
- _Age:_ idade do passageiro (em anos)
- _Sibsp:_ número de irmãos/cônjuges à bordo
- _Parch:_ número de pais/filhos à bordo
- _Ticket:_ número do bilhete
- _Fare:_ valor do bilhete
- _Cabin:_ número da cabine
- _Embarked:_ porto de embarque (C = Cherbourg, Q = Queenstown, S = Southampton)

In [ ]:
dados_treinamento = pd.read_csv('./train.csv')
dados_teste = pd.read_csv('./test.csv')

print('Shape treinamento:', dados_treinamento.shape)
print('Shape teste:', dados_teste.shape)

É perceptível que os dados de teste possuem 1 coluna à menos que os dados de treinamento. Essa coluna é justamente a _Survived_ a ser predita.

## Limpeza, Conversão e Padronização dos Dados

### Remoção de Atributos Irrelevantes

Alguns atributos não possuem influência na sobrevivência dos passageiros, como _PassengerId_, _Name_ e _Ticket_. Tais atributos podem ser removidos dos datasets.

In [ ]:
ids_teste = dados_teste['PassengerId']

dados_treinamento.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
dados_teste.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

### Tratamento de Valores Nulos

A fim de melhorar a integridade dos dados lidos, é necessário verificar a melhor maneira de se tratar valores nulos nos datasets.

In [ ]:
dados_treinamento.isnull().sum().sort_values(ascending=False)

In [ ]:
dados_teste.isnull().sum().sort_values(ascending=False)

Percebe-se que o atributo _Cabin_ é o que possui mais registros com valores nulos (em ambos os datasets). Por se tratar de um atributo complementar, sem tanta influência projetada, é possível descartar a coluna.

In [ ]:
dados_treinamento.drop(['Cabin'], axis=1, inplace=True)
dados_teste.drop(['Cabin'], axis=1, inplace=True)

Já os atributos _Age_, _Embarked_ e _Fare_ parecem ser mais determinantes para a classificação e a análise deve ser um pouco mais cautelosa.

Para _Age_, os valores nulos podem ser preenchidos com a média dos não nulos.

In [ ]:
dados_treinamento['Age'].fillna(dados_treinamento['Age'].mean(), inplace=True)
dados_teste['Age'].fillna(dados_teste['Age'].mean(), inplace=True)

Para _Embarked_, como se tratam de apenas 2 registros, os valores nulos podem ser substituídos para o porto mais provável (S = Southampton).

In [ ]:
dados_treinamento['Embarked'].hist(figsize=(4,4));
dados_treinamento['Embarked'].fillna('S', inplace=True)

Para _Fare_, o único valor nulo pode ser preenchdo com a mediana dos não nulos, uma medida não sensível à outliers.

In [ ]:
dados_teste['Fare'].fillna(dados_teste['Age'].median(), inplace=True)

### Conversão de Atributos Nominais

Os atributos nominais dos datasets podem ser convertidos para numéricos via one-hot-encoding, já implementado pelo _pandas_.

In [ ]:
dados_treinamento_com_nominais = dados_treinamento

dados_treinamento = pd.get_dummies(dados_treinamento)
dados_teste = pd.get_dummies(dados_teste)

dados_treinamento

### Transformação dos Dados para Classificação

Os dados de treinamento e de teste podem ser convertidos para matrizes do _numpy_, de forma a facilitar o processamento na classificação.

In [ ]:
dados_treinamento_np = dados_treinamento.to_numpy()

Y_treinamento = dados_treinamento_np[:,0]   # linhas da coluna survived
X_treinamento = dados_treinamento_np[:,1:]  # linhas das demais colunas

In [ ]:
dados_teste_np = dados_teste.to_numpy()

X_teste = dados_teste

### Padronização dos Dados

In [ ]:
X_treinamento = StandardScaler().fit_transform(X_treinamento)
X_teste = StandardScaler().fit_transform(X_teste)

## Análise Descritiva dos Dados

É possível plotar uma Matriz de Correlação entre os atributos.

In [ ]:
corr_treinamento = dados_treinamento.corr()

plt.figure(figsize=(7,7))
plt.imshow(corr_treinamento, cmap='Blues')
plt.colorbar()
plt.xticks(range(len(corr_treinamento)), corr_treinamento.columns, rotation='vertical');
plt.yticks(range(len(corr_treinamento)), corr_treinamento.columns);
plt.title('Correlação entre as Variáveis')
plt.show()

Pela Matriz de Correlação atributo _Survived_ está mais positivamente relacionado com _Sex\_female_, _Fare_ e _Embarked\_C_.

In [ ]:
sns.catplot(x='Sex', hue='Survived', kind='count', data=dados_treinamento_com_nominais);

Enquanto cerca de 75% das mulheres sobreviveram, apenas cerca de 20% tiveram a vida preservada.

In [ ]:
dados_treinamento_com_nominais['Fare_Range'] = pd.qcut(dados_treinamento_com_nominais['Fare'], 4)

sns.barplot(x='Fare_Range', y='Survived', data=dados_treinamento_com_nominais);

Quanto maior o preço do bilhete, maior o percentual de sobreviventes.

In [ ]:
sns.catplot(x='Embarked', hue='Survived', kind='count', data=dados_treinamento_com_nominais);

Mais da metade dos passageiros que embarcaram em Cherbourg sobreviveram, quase metade dos que embarcaram em Queenstown sobreviveram e a menor parte (cerca de 1/3) dos que embarcaram em Southampton sobreviveram.

Ainda pela Matriz de Correlação, o atributo _Survived_ está mais negativamente relacionado com _Sex\_male_ (atributo complementar ao _Sex\_female_ já apresentado), _Pclass_, _Embarked\_S_ (atributo complementar ao _Embarked\_C_ já apresentado) e _Age_.

In [ ]:
sns.heatmap(dados_treinamento_com_nominais.groupby(['Pclass', 'Survived']).size().unstack(), annot=True, fmt='d');

A maioria dos passageiros que embarcaram na primeira classe sobreviveu. Na segunda classe, a distribuição foi equilibrada. Já para a terceira classe, a maioria (75%) dos passageiros não sobreviveu.

In [ ]:
sns.violinplot(x='Sex', y='Age', hue='Survived', data=dados_treinamento_com_nominais, split=True);

A maior parte das crianças masculinas sobreviveu, mas as crianças femininas não conseguiram a sobrevivência em sua maioria.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
componentes_principais = pca.fit(X_treinamento)

variancia_explicada = pca.explained_variance_
razao_variancia_explicada = pca.explained_variance_ratio_
razao_variancia_acumulada = np.cumsum(razao_variancia_explicada)

numero_componentes = len(variancia_explicada)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(range(1,numero_componentes + 1), razao_variancia_explicada, alpha=0.8, ec='black', align='center')
plt.title('Variância Explicada por Componente', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Número de Componentes', fontsize=12)
plt.ylabel('Proporção de Variância Explicada', fontsize=12)

plt.subplot(1, 2, 2)
plt.plot(range(1,numero_componentes + 1), razao_variancia_acumulada, marker='o', linestyle='--', color='r')
plt.title('Variância Explicada Acumulada', fontdict={'size': 12, 'weight': 'bold'})
plt.xlabel('Número de Componentes', fontsize=12)
plt.ylabel('Proporção Acumulada de Variância Explicada', fontsize=12)

plt.tight_layout()
plt.show()

## Seleção de Modelos e Classificação

### K Nearest Neighbors

In [ ]:
parametros_grid = {
    'n_neighbors': [k for k in range(50) if k % 2 != 0],
    'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']   
}

modelo = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=parametros_grid,
    cv=10,
    scoring='roc_auc',
    return_train_score=False,
    verbose=True,
    error_score='raise'
)

modelo.fit(X_treinamento, Y_treinamento)
print(modelo.best_params_)
modelo.score(X_treinamento, Y_treinamento)

In [ ]:
Y_previsto = np.array(modelo.predict(X_teste), dtype=int)

dados_previsao = pd.DataFrame()
dados_previsao['PassengerId'] = ids_teste
dados_previsao['Survived'] = Y_previsto

dados_previsao.to_csv('prediction-knn.csv', index=False)

Submetendo no Kaggle, foi obtido um score de **0.75358**

### Random Forest




In [ ]:
parametros_grid = {
    'n_estimators': [i * 100 for i in range(1, 4)],
    'max_depth': [i * 5 for i in range(1, 6)],
    'criterion': ['gini', 'entropy'],
}

modelo = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=parametros_grid,
    cv=10,
    scoring='roc_auc',
    n_jobs=-1
)

modelo.fit(X_treinamento, Y_treinamento)
print(modelo.best_params_)
modelo.score(X_treinamento, Y_treinamento)

In [ ]:
Y_previsto = np.array(modelo.predict(X_teste), dtype=int)

dados_previsao = pd.DataFrame()
dados_previsao['PassengerId'] = ids_teste
dados_previsao['Survived'] = Y_previsto

dados_previsao.to_csv('prediction-random-forest.csv', index=False)

Submetendo no Kaggle, foi obtido um score de **0.79186**

### Regressão Logística

In [ ]:
parametros_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga'],
}

modelo = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=parametros_grid,
    cv=10,
    scoring='roc_auc',
    n_jobs=-1,
    error_score='raise'
)

modelo.fit(X_treinamento, Y_treinamento)
print(modelo.best_params_)
modelo.score(X_treinamento, Y_treinamento)

In [ ]:
Y_previsto = np.array(modelo.predict(X_teste), dtype=int)

dados_previsao = pd.DataFrame()
dados_previsao['PassengerId'] = ids_teste
dados_previsao['Survived'] = Y_previsto

dados_previsao.to_csv('prediction-logistic.csv', index=False)

Submetendo no Kaggle, foi obtido um score de **0.77511**

### Multilayer Perceptron

In [ ]:
parametros_grid = {
    'hidden_layer_sizes': [(100,), (50,50), (30,30,30), (10,20,30), (30,20,10)],
    'activation': ['relu', 'logistic', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

modelo = GridSearchCV(
    estimator=MLPClassifier(max_iter=1000),
    param_grid=parametros_grid,
    cv=10,
    n_jobs=-1
)

modelo.fit(X_treinamento, Y_treinamento)
print(modelo.best_params_)
modelo.score(X_treinamento, Y_treinamento)

In [ ]:
Y_previsto = np.array(modelo.predict(X_teste), dtype=int)

dados_previsao = pd.DataFrame()
dados_previsao['PassengerId'] = ids_teste
dados_previsao['Survived'] = Y_previsto

dados_previsao.to_csv('prediction-perceptron.csv', index=False)

Submetendo no Kaggle, foi obtido um score de **0.76315**

## Comentários Finais

Em resumo, os scores obtidos por cada modelo no conjunto de teste foram:

| Classificador | Parâmetros | Score | 
|:-------------:|:-----:|:----------:|
| KNN | Distância Manhattan, 13 Vizinhos | 75,35% | 
| Random Forest | Critério Gini, 5 de Profundidade Máxima, 100 Estimadores | 79,18% | 
| Regressão Logística | C 0.01, Solver lbfgs | 77,51% |
| Multilayer Perceptron | Ativação Relu, $\alpha$ 0.001, Camadas (100), Taxa de Aprendizado Constante  | 76,31% |